In [1]:
import csv
import os
import cv2

folder_path = "./Dataset/"
classes = {'NOC': 0,
           'OSCC': 1
           }


def load_images(directory):
    A, B = [], []
    for r, d, f in os.walk(folder_path + directory):
        for file in f:
            img = cv2.imread(r + "/" + file)[...,::-1]
            img = cv2.resize(img, (224, 224))
            if(classes[r[16:]]==0):
                A.append(img)
            else:
                B.append(img)

    return A, B

In [2]:
dataTest0, dataTest1 = load_images('testq')
# dataTr0, dataTr1 = load_images('train')

In [3]:
B = [[0, item] for item in dataTest0]
B = B + [[1, item] for item in dataTest1]

# A = [[0, item] for item in dataTr0]
# A = A + [[1, item] for item in dataTr1]

In [4]:
import numpy as np

y = [[1, 0], [0, 1]]

X_val = np.array([item[1] for item in B]).astype(float) / 255.0
Y_val = np.array([y[int(item[0])] for item in B]).astype(float)
Y_val = Y_val.reshape(305, 2)

# X_val1 = np.array([item[1] for item in A]).astype(float) / 255.0
# Y_val1 = np.array([y[int(item[0])] for item in A]).astype(float)
# Y_val1 = Y_val1.reshape(1682, 2)

In [5]:
Y_val.shape

(305, 2)

In [6]:
X_val.shape

(305, 224, 224, 3)

In [7]:
from keras.models import model_from_json

with open("./Final Model/model1.json", "r") as f:
    model1 = model_from_json(f.read())
    f.close()
    
model1.load_weights("./Final Model/modelOpt1.h5")

with open("./Final Model/model2.json", "r") as f:
    model2 = model_from_json(f.read())
    f.close()
    
model2.load_weights("./Final Model/modelOpt2.h5")

with open("./Final Model/model3.json", "r") as f:
    model3 = model_from_json(f.read())
    f.close()
    
model3.load_weights("./Final Model/modelOpt3.h5")

In [8]:
y1 = model1.predict(X_val)
y2 = model2.predict(X_val)
y3 = model3.predict(X_val)


10/10 [==============================] - 34s 3s/step


In [9]:
y = (1*y1+6*y2+4*y3)/(11)

In [10]:
confMatrix1 = [[0, 0], [0, 0]]
confMatrix2 = [[0, 0], [0, 0]]
confMatrix3 = [[0, 0], [0, 0]]
confMatrix = [[0, 0], [0, 0]]

for i in range(len(y)):
    confMatrix[Y_val[i].argmax()][y[i].argmax()] += 1
    confMatrix1[Y_val[i].argmax()][y1[i].argmax()] += 1
    confMatrix2[Y_val[i].argmax()][y2[i].argmax()] += 1
    confMatrix3[Y_val[i].argmax()][y3[i].argmax()] += 1

In [11]:
print(confMatrix1)
print(confMatrix2)
print(confMatrix3)
print(confMatrix)

[[57, 15], [17, 216]]
[[59, 13], [13, 220]]
[[63, 9], [30, 203]]
[[60, 12], [10, 223]]


In [36]:
def evaluate_model(CM):
    t = len(y)

    acc = (CM[0][0]+CM[1][1])/t
    pre = (CM[1][1])/(CM[0][1]+CM[1][1])
    reca = (CM[1][1])/(CM[1][0]+CM[1][1])
    f1 = 2*pre*reca/(pre+reca)
    # specificity = confMatrix[0][0]/(confMatrix[0][0]+confMatrix[1][0])
    
    return([acc, pre, reca, f1])

In [38]:
print(evaluate_model(confMatrix))
print(evaluate_model(confMatrix1))
print(evaluate_model(confMatrix2))
print(evaluate_model(confMatrix3))

[0.9278688524590164, 0.948936170212766, 0.9570815450643777, 0.952991452991453]
[0.8950819672131147, 0.935064935064935, 0.927038626609442, 0.9310344827586207]
[0.9147540983606557, 0.944206008583691, 0.944206008583691, 0.944206008583691]
[0.8721311475409836, 0.9575471698113207, 0.871244635193133, 0.9123595505617977]


In [14]:
y[240], Y_val[240]

(array([0.3646447, 0.6353553], dtype=float32), array([0., 1.]))

In [15]:
# Try to find perfect weights

In [16]:
print(y.shape, y1.shape)

(305, 2) (305, 2)


In [17]:
yq = Y_val[:,1]

In [18]:
# import math
# w = []
# bestLoss = 100000

# for w1 in range(1, 100):
#     for w2 in range(1, 100):
#         for w3 in range (1, 100):
#             if(w1==0 and w2==0 and w3==0):
#                 continue
#             yt = ( (w1*y1+w2*y2+w3*y3)/float(w1+w2+w3))[:,1]
#             # loss = (1. / yq.shape[0]) * (-np.dot(yq, np.log(yt)) - np.dot(1 - yq, np.log(1 - yt)))
#             loss = 0
#             loss1 = 0
#             for i in range(305):
#                 if (yq[i]==1 and yt[i]<0.5):
#                     loss+=1
#                 elif (yq[i]==0 and yt[i]>=0.5):
#                     loss1+=1 
#             if(loss < bestLoss):
#                 bestLoss = loss
#                 w = [w1, w2, w3]
#                 print(w, bestLoss, loss1)

# print(w, bestLoss)
